### Dictionary

train_direction = 0 south, 1 north
train_type = 0 Local, 1 Limited, 2 Bullet
train_

In [1]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import re
import random
import operator

from func import *

# inline plot
%matplotlib inline
#%%javascript
#IPython.OutputArea.auto_scroll_threshold = 9999;

/Users/albarron/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#%load 'data/raw-twt2016-01-26-14/21/09.csv'
df = pd.read_csv("data/raw-twt2016-01-26-14-21-09.csv",sep='\t',error_bad_lines=False)
# df.head(5)
print len(df.index)
list(df.columns.values)

3199


['Unnamed: 0',
 'created_at',
 'favorite_count',
 'favorited',
 'hashtags',
 'id',
 'in_reply_to_screen_name',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'lang',
 'media',
 'place',
 'retweet_count',
 'retweeted',
 'retweeted_status',
 'source',
 'text',
 'truncated',
 'urls',
 'user',
 'user_mentions']

Cleanin' the data

In [3]:
# Fill in blank hashtags
df = df.where((pd.notnull(df)), np.nan)
df["hashtags"].fillna('')

# Add some date/time things
df["created_at"] = pd.to_datetime(df["created_at"], errors='coerce')

df["day_of_week"] = df["created_at"].apply(lambda x: x.weekday())
df["day_of_month"] = df["created_at"].apply(lambda x: x.day)
df["month"] = df["created_at"].apply(lambda x: x.month)
df["isRushHour"] = df["created_at"].apply(lambda x: get_time_of_day(x))

# del tod_Dummy['shutdown']

# df['in_reply_to_screen_name'].fillna(-1)
# df['in_reply_to_status_id'].fillna(-1)
# df['in_reply_to_user_id'].fillna(-1)
# df['retweeted_status'].fillna(-1)
# df['retweeted'].fillna(-1)
df['retweet_count'].fillna(np.nan)
df['favorite_count'].fillna(np.nan)
df["hashtags"].fillna(np.nan)
df["hashtags"] = df["hashtags"].apply(lambda x: str(x)[1:-1])
df.loc[df["hashtags"]=='a',"hashtags"] = ''
#list(df.columns.values)

   error  rush_evening  rush_morning  workday
0      1             0             0        0
1      0             1             0        0
2      0             1             0        0
3      0             1             0        0
4      0             1             0        0
error           3199
rush_evening    3199
rush_morning    3199
workday         3199
dtype: int64


In [4]:
#Potentially remove, just cleaning for analysis sake
del df['Unnamed: 0']
del df['truncated']
del df['user_mentions']
del df['urls']
del df['source']
del df['lang']
del df['place']
del df['favorited']
del df['media']
del df['user']

# More likely to remove
del df['in_reply_to_status_id']
del df['in_reply_to_user_id']
del df['retweeted']
del df['retweeted_status']
len(df)

3199

Let's start getting some more detailed data from the trips as well

In [5]:
# df['favorite_count'] = df['favorite_count'].astype(np.int64)
# df['retweet_count'] = df['retweet_count'].astype(np.int64)
# df['text'] = df['text'].astype(str)
# df['id'] = df['id'].astype(np.int64)
# df['day_of_week'] = df['day_of_week'].astype(np.int64)
# df['day_of_month'] = df['day_of_month'].astype(np.int64)
# df['month'] = df['month'].astype(np.int64)
# df['time_of_day'] = df['time_of_day'].astype(np.int64)
df.loc[df["hashtags"]=='on',"hashtags"] = np.nan
df.convert_objects(convert_numeric=True)
df.dtypes
len(df)

/Users/albarron/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


3199

In [6]:
# Pull out potential trains from both hashtags and text
df["topic_train"] = df["text"].apply(lambda x: check_train_id(x))
df["topic_train"] = df["topic_train"].apply(lambda x: str(x)[1:-1])
df["topic_train"].fillna(np.nan)
df.head(2)

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,retweet_count,text,day_of_week,day_of_month,month,time_of_day,topic_train
0,2016-01-26 20:32:15,6,SanFrancisco,692082643022680064,NaN,7,NOTICE: Ped &amp; Bike detours in place for Ma...,1,26,1,error,
1,2016-01-26 19:41:32,NaN,,692069881559134208,therealwall,NaN,@therealwall After the end of the concert we w...,1,26,1,rush_evening,


First, a word about the below code.
In the accompanying `func.py` there is a function called parse_train that returns a `pandas.Series` object. For some reason, when it's returned from a map or apply, it seems to get cast as a string. When applied to a list or a dataframe, this string gets turned into a single field in the row, OR divided into several rows, throwing the count off.

To get around this, I return the results of the parse_train function and then CAST it back to a series. This adds a weird 0 index, which I delete. I then fill in the plethora of NaNs and recombine it with the primary dataframe.

For context, previous iterations included
`df['topic_train'].apply(lambda x:parse_train(x))`
which would return a pd.Series object with `str` versions of the returned pd.Series from `parse_train`

In [7]:
ret = []

def parse_train(t):
# Revised this function to work with categorical variables
# x should be a list with train codes eg 123
# {"id": "123", "type:" "bullet", direction: "south"}

    try:
        s = t['topic_train'].split(',')
    except:
        return t['topic_train']
    if s[0] == '':
        return np.nan
    for x in s:
        q = {}
        x = str(x)
        x = re.sub('[^0-9]','', x)
        if len(x)<3: continue

        # 1 = north, 0 = south
        q["t_northbound"] = 1 if int(x[2]) in [1,3,5,7,9] else 0
        q['t_limited'] = 0
        q['t_bullet'] = 0
        
        if x[0] == '1':
            q['t_limited'] = 0
        elif x[0] == '2':
            q["t_limited"] = 1 # limited
        elif x[0] == '3':
            q["t_bullet"] = 1 # bullet
        else:
            q['t_limited'] = 0

        ret.append({'tweet_id': t['id'],
                    'timestamp': t['created_at'], 
                    'train_id': int(x),
                    't_northbound':q["t_northbound"], 
                    't_limited': q["t_limited"],
                    't_bullet': q['t_bullet']})
    return s

In [8]:
# Let's then filter those train topics into details
# Btw this is jank as fuck.

# red = df[['id','created_at','topic_train']]
red = df.apply(lambda x:parse_train(x),axis=1)
print "red return:",len(red)
print "ret return,",len(ret)
#red
tf = pd.DataFrame(ret)
tf.head(5)

#events = pd.DataFrame([pd.Series(x) for x in red.apply(parse_train)])
#events
#del new.iloc[0]
#new.fillna('')
#df.combine_first(new)

red return: 3199
ret return, 528


,t_bullet,t_limited,t_northbound,timestamp,train_id,tweet_id
0,0,0,1,2016-01-25 23:42:14,155,691768068385718275
1,0,0,1,2016-01-22 22:48:57,151,690667494906814464
2,0,0,1,2016-01-20 22:22:55,151,689936168893329408
3,0,0,0,2016-01-19 18:08:25,138,689509733640732672
4,0,1,0,2016-01-19 16:32:04,216,689485484838416388


In [9]:
print df.loc[df['topic_train'] != '',['topic_train','text']]

              topic_train                                               text
2                    '50'  @AemalTheAFGHAN @BKDenverSports We're glad to ...
8                    '50'  @iamsridhar we are adding extra trains &amp; c...
16                  '155'                  #NB155 is 22 mins late. #Caltrain
21                   '50'  RT @smctd: #Caltrain Devs: NEW Super Bowl sche...
22               '50', ''  Headed to #SantaClara for #SB50? Visit our SB ...
29                   '50'  RT @Broncos: We #BeatThePatriots!\n\nSee you i...
30                   '50'  We look forward to getting #CAR @Panthers, #DE...
32                  '151'        #NB 151 is running 10 mins behind #Caltrain
33                     ''  Trains 236-254 will board on the NB platform t...
44                  '151'     #NB151 is 10 mins down at San Mateo. #Caltrain
51                  '138'  #SB138 will board on the northbound platform a...
55    '216', '225', '329'  #SB216 delayed 13 minutes at SAT\n#NB225 delay...

In [10]:
len(tf)

528

In [11]:
# Merge on tweet ID,
df = df.merge(tf, left_on='id',right_on='tweet_id',how='right')

In [ ]:
# Okay, let's try and get a yae or nae to delay and see our hit rate.

In [12]:
# Only events that have train mentioned
trains = df[df['train_id'] > 0]

In [16]:
# d
filename = "./data/formated_twts.csv"
df.to_csv(filename, sep=',', encoding='utf-8')